In [4]:
import pandas as pd

# -----------------------------------
# 0. 파일 경로 설정
# -----------------------------------
in_path  = "./../cohort/ecg_ver50.csv"   # 입력 파일
out_path = "./../cohort/ecg_ver50_stemi_flag.csv"        # 결과 저장 파일

# -----------------------------------
# 1. 데이터 로딩
# -----------------------------------
df = pd.read_csv(in_path, low_memory=False)

# -----------------------------------
# 2. STEMI 진단 flag 생성
#    - report_0 ~ report_17 에서 텍스트 검색
#    - NSTEMI / NON-ST elevation 등은 제외
# -----------------------------------
# report 컬럼 리스트
report_cols = [c for c in df.columns if c.startswith("report_")]

def detect_stemi_from_reports(row) -> int:
    # report 텍스트를 하나로 합치고 소문자로 통일
    texts = []
    for c in report_cols:
        val = row.get(c)
        if pd.notna(val):
            texts.append(str(val))
    full_text = " ".join(texts).lower().strip()

    if not full_text:
        return 0

    # 1) NSTEMI / non-ST 관련 표현은 먼저 제외
    if "nstemi" in full_text:
        return 0
    if "non st" in full_text or "non-st" in full_text or "nonst" in full_text:
        return 0
    if "non st elevation" in full_text or "non-st elevation" in full_text:
        return 0

    # 2) STEMI / ST elevation MI 패턴 탐지
    #    - 예: "*** CONSIDER ACUTE ST ELEVATION MI ***"
    if "stemi" in full_text:
        return 1
    if ("st elevation" in full_text or "st-elevation" in full_text) and "mi" in full_text:
        return 1
    if "acute st segment elevation" in full_text:
        return 1

    return 0

# row 단위로 flag 계산
df["stemi_flag"] = df.apply(detect_stemi_from_reports, axis=1)

print("[INFO] STEMI flag 분포")
print(df["stemi_flag"].value_counts(dropna=False))

# -----------------------------------
# 3. report 및 기타 ECG 세부 컬럼 삭제
# -----------------------------------
drop_cols = report_cols + [
    "bandwidth",
    "filtering",
    "rr_interval",
    "p_onset",
    "p_end",
    "qrs_onset",
    "qrs_end",
    "t_end",
    "p_axis",
    "qrs_axis",
    "t_axis",
]

# 실제 존재하는 컬럼만 삭제 대상에 포함
drop_cols = [c for c in drop_cols if c in df.columns]

df = df.drop(columns=drop_cols)

# -----------------------------------
# 4. 저장
# -----------------------------------
df.to_csv(out_path, index=False)
print(f"[SAVE] 저장 완료: {out_path}")


[INFO] STEMI flag 분포
stemi_flag
0    25276
1     2205
Name: count, dtype: int64
[SAVE] 저장 완료: ./../cohort/ecg_ver50_stemi_flag.csv
